In [1]:
from importlib import reload
import LambdaRankHW
import numpy as np
import query
import os
import matplotlib.pyplot as plt
import pandas as pd

DEBUG: nvcc STDOUT nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mod.cu
   Creating library C:/Users/Dana/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-3.5.2-64/tmpys3ncrrd/m91973e5c136ea49268a916ff971b7377.lib and object C:/Users/Dana/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-SP0-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-3.5.2-64/tmpys3ncrrd/m91973e5c136ea49268a916ff971b7377.exp

Using gpu device 0: GeForce 940MX (CNMeM is enabled with initial size: 80.0% of memory, CuDNN 5110)
C:\Users\Dana\Anaconda3\lib\site-packages\theano\sandbox\cuda\__init__.py:600: UserWarning: Your CuDNN version is more recent then Theano. If you see problems, try updating Theano or downgrading CuDNN to version 4.
  warnings.warn(warn)
C:\Users\Dana\Anaconda3\lib\site-packages\theano\tensor\signal\do

In [ ]:
train_queries = query.load_queries(os.path.normpath('./HP2003/Fold1/train.txt'), 64)
test_queries = query.load_queries(os.path.normpath('./HP2003/Fold1/test.txt'), 64)
val_queries = query.load_queries(os.path.normpath('./HP2003/Fold1/vali.txt'), 64)

In [34]:
%%prun
reload(LambdaRankHW)
n_epochs = 20
measure_type = LambdaRankHW.LISTWISE
num_features = 64
num_folds = 1

res = LambdaRankHW.experiment(n_epochs, measure_type, num_features, num_folds)


Loading train queries
Loading val queries
Creating the S Matrix
input_dim 64 output_dim 1
finished create_iter_functions
Epoch 1 of 20 took 3.210s
training loss:		-37.577946
training mNDCG:		0.791596
validation mNDCG:		0.673161

Epoch 10 of 20 took 28.704s
training loss:		-0.896609
training mNDCG:		0.833720
validation mNDCG:		0.719352

Epoch 20 of 20 took 31.969s
training loss:		-0.385707
training mNDCG:		0.856835
validation mNDCG:		0.757054

 

In [ ]:
valndcgs = [[rr['val_mndcg'] for rr in res[i][:-1]] for i in res]  
trainndcgs = [[rr['train_mndcg'] for rr in res[i][:-1]] for i in res]  

df = pd.DataFrame(valndcgs, index=['Val Fold%d' % d for d in range(1, num_folds + 1)]).T
plt.plot(df)
df2 = pd.DataFrame(trainndcgs, index=['Train Fold%d' % d for d in range(1, num_folds + 1)]).T
plt.plot(df2)
plt.legend(df2.columns + df.columns)
plt.show()

In [5]:
# Calculates the best NDCG@k for query with r relevant documents and binary relevance labels
def best_ndcg(r, k):
    if r == 0:
        raise ZeroDivisionError("No relevant documents for given query. NDCG can not be computed.")
    sum_limit = min(r, k)
    b_rank = 1 / np.log(1 + np.array(range(1, sum_limit + 1)))
    return b_rank, np.cumsum(b_rank, axis=0)

# Store the logarithmic discount and normalization factor lists for faster NDCG computation
disc_list, norm_list = best_ndcg(1000,1000)
disc_list = disc_list#.reshape((-1,1))

# Calculates the NDCG@k for a rank with binary relevance labels assuming a query with r relevant documents
def ndcg(rank, k, r = 1):
    if r == 0:
        #raise ZeroDivisionError("No relevant documents for given query. NDCG can not be computed.")
        return 0
    return np.transpose(rank[:k]).dot(disc_list[:k]) / norm_list[r-1]


In [10]:
print(np.cumsum(disc_list)[:10])
print(norm_list[:10])

[ 1.44269504  2.35293427  3.07428179  3.69561672  4.25372735  4.76762569
  5.24852404  5.70364365  6.13793813  6.55497053]
[ 1.44269504  2.35293427  3.07428179  3.69561672  4.25372735  4.76762569
  5.24852404  5.70364365  6.13793813  6.55497053]


In [3]:
def delta_ndcdg(i, j, id_i, id_j, labels):
    return (2**labels[id_i] - 2**labels[id_j]) * (disc_list[j] - disc_list[i]) / norm_list[int(np.sum(labels))-1]

In [7]:
doc_ids = np.array(range(1,1001))
labels =np.ones(1000)
scores =np.random.random(1000)
w = 1
l = 2

def meh():
    order = np.argsort(-scores)
    ordered_ids = doc_ids[order]
    # print(order)
    # print(ordered_ids)
    delta_val = np.abs(delta_ndcdg(np.where(ordered_ids == w)[0], np.where(ordered_ids == l)[0], w, l, labels))

%timeit meh ()   

r1 = np.array(labels)[order]
r2 = np.array(labels)[order]
aux = r2[np.where(ordered_ids == w)[0]]
r2[np.where(ordered_ids == w)[0]] = r2[np.where(ordered_ids == l)[0]]
r2[np.where(ordered_ids == l)[0]] = aux
diff = np.abs(ndcg(r2, len(labels), sum(labels)) - ndcg(r1,len(labels),sum(labels)))
# if (diff != delta_val):
print(delta_val, diff)

The slowest run took 6.52 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 56.6 µs per loop


C:\Users\Dana\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


NameError: name 'delta_val' is not defined